# Compter les répétitions avec YOLO v8


## Introduction

Dans le monde du fitness et de l'entraînement personnel, le suivi des répétitions d'exercices est essentiel pour suivre les progrès, fixer des objectifs et s'assurer que la forme est correcte. Si le comptage manuel des répétitions peut s'avérer fastidieux et source d'erreurs, les techniques modernes de computer vision offrent une solution plus précise et plus pratique. Le modèle YOLOv8-Pose : YOLOv8 représente la dernière version de l'algorithme de détection d'objets You Only Look Once (YOLO), développé par Ultralytics. En plus de la détection d'objets, YOLOv8 inclut une branche dédiée à l'estimation de pose, capable de détecter 17 points clés du corps humain.  Cette capacité en fait un choix idéal pour le suivi d'exercices impliquant des mouvements et des positions spécifiques du corps. 

<img src="picture/288088210-f45d8315-b59f-47b7-b9c8-c61af1ce865b.png" alt="YOLO v8 Keypoints" width="600" height="600">





## Notre approche
L'application de comptage de répétitions que nous présentons ici utilise le modèle YOLOv8-Pose pour détecter les points clés du corps humain pendant les exercices. En identifiant ces points clés et en calculant les angles entre les lignes formées par certains points, nous pouvons déterminer quand une répétition est complétée. 
Par exemple, pour un squat, nous suivons l'angle entre les points de la hanche, du genou et de la cheville pour identifier le moment où l'utilisateur atteint la position la plus basse avant de revenir à la position de départ, ce qui compte comme une répétition.
<img src="picture/upload_2018-9-4_0-46-20.png" alt="SQUAT Angle" width="300" height="400">



In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

def calculate_angle(p1, p2, p3):
   # p1, p2, p3 sont les points au format [x, y]
    # Calculer les vecteurs
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p2)

    # Calculer l'angle en radians
    angle_rad = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

    # Convertir en degrés
    angle_deg = np.degrees(angle_rad)

    return angle_deg

Pour le choix du modèle, nous opterons pour un modèle préentraîné de YOLOv8 sur le jeu de données COCO. Ce jeu de données contient 200 000 images annotées avec des points clés, ce qui en fait une référence de choix pour les tâches d'estimation de la pose. Nous avons décidé d'utiliser un modèle préentraîné pour plusieurs raisons.

Tout d'abord, le temps et les ressources nécessaires pour générer un jeu de données de cette ampleur sont considérables. La collecte et l'annotation manuelle de centaines de milliers d'images avec des points clés exigent une main-d'œuvre significative et spécialisée. De plus, la qualité des annotations doit être rigoureuse pour garantir la précision du modèle d'estimation de la pose, ce qui rend cette tâche encore plus ardue.

En utilisant un modèle préentraîné sur COCO, nous bénéficions d'un ensemble de données riche et diversifié, avec des annotations de haute qualité réalisées par des experts. Cela nous permet de nous concentrer sur l'application et l'optimisation du modèle pour notre cas d'utilisation spécifique, sans avoir à passer par les étapes longues et coûteuses de création et d'annotation de données.

Ainsi, cette approche nous permet de gagner du temps et des ressources, tout en nous assurant de disposer d'un modèle performant pour l'estimation de la pose, grâce à l'expertise et à la vaste quantité de données déjà disponibles dans le jeu de données COCO.

In [2]:
model = YOLO("yolov8n-pose.pt")

Ensuite, il nous faudra une fonction pour vérifier si la posture de l'exercice de squat est correcte. Pour ce faire, nous calculerons l'angle de la jambe à partir des points clés de la hanche, du genou et de la cheville. Parallèlement, nous déterminerons l'angle du corps en utilisant les points clés de l'épaule, de la hanche et du genou.

Pour la position de départ, nous devons nous assurer que les angles de la jambe et du corps sont supérieurs à 160 degrés. En ce qui concerne la position finale, les angles doivent être inférieurs à 110 degrés.

Pour gérer la transition entre ces deux positions, nous utiliserons un booléen appelé transition. Ce booléen nous permettra de vérifier que l'utilisateur passe correctement de la position de départ à la position finale et inversement, garantissant ainsi la complétion correcte de chaque répétition.

In [3]:
def check_form(keypoints, transition):
    f = open("demofile0.txt", "a")
    leg_angle = calculate_angle(keypoints[0][11], keypoints[0][13], keypoints[0][15])
    body_angle = calculate_angle(keypoints[0][5], keypoints[0][11], keypoints[0][13])


    if leg_angle > 160 and not transition:
        if body_angle > 160:
            return True
        else:
            f.write(f"Body Angle: {body_angle}\n")
            f.close()
            return False


    elif leg_angle < 120 and not transition:
        if body_angle < 110:
            return True
        else:
            f.write(f"Body Angle: {body_angle}\n, Leg Angle: {leg_angle}\n")
            f.close()
            return False


    elif 120 <= leg_angle <= 160 and transition:
        if 110 <= body_angle <= 160:
            return True


Nous allons ensuite intégrer tout cela dans une boucle principale où nous appliquerons notre modèle YOLOv8 à une vidéo ou à une caméra pour détecter les points clés et les afficher. Nous utiliserons la même logique que celle utilisée dans la fonction check_form, cette fois en introduisant un booléen is_down pour déterminer si le mouvement va vers le bas ou vers le haut. Nous compterons le nombre de répétitions effectuées et mettrons également à jour notre booléen transition pour qu'il soit pris en compte par notre fonction check_form.

Nous afficherons les résultats directement sur l'image et indiquerons à l'utilisateur de corriger sa posture si nécessaire. Cette approche nous permettra de suivre et d'analyser la qualité des mouvements réalisés lors de l'exercice, tout en fournissant un retour immédiat pour aider à maintenir une bonne technique.

In [4]:
squat_count = 0
transition =False
is_down = False
cap = cv2.VideoCapture("squat.mp4")
# cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    try:
        success, frame = cap.read()

        if success:
            results = model(frame)
            for r in results[0]:
                keypoints = r.keypoints.xy
                angle = calculate_angle(keypoints[0][11], keypoints[0][13], keypoints[0][15])
                print("Form: ", check_form(keypoints,transition))
                print("Angle: ", angle)
                Body_angle = calculate_angle(keypoints[0][5],keypoints[0][11],keypoints[0][13])
                print("Body: ", Body_angle)
                print(transition)
                if angle > 160 and is_down == False :
                    is_down = True
                elif angle > 160 and is_down == True and transition == True :
                    transition = False

                elif 120 < angle < 160 and transition == False:
                    transition = True

                elif angle < 120 and is_down == False and transition == True:
                    transition = False

                elif angle < 120 and is_down == True :
                    is_down = False
                    squat_count += 1

                print('squat count:', squat_count)
                form_correct = check_form(keypoints, transition)
                if not form_correct:
                    cv2.putText(frame, 'Correct the Posture', (800, 100), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 255), 10)

            annotated_frame = results[0].plot()

            cv2.putText(annotated_frame, f'Squats: {squat_count}', (20, 70), 2, 2, (10, 10, 255), 5)
            cv2.imshow("Inference",annotated_frame)


            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    except ValueError:
        print("Get into position !")


cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 161.8ms
Speed: 6.5ms preprocess, 161.8ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)
Form:  True
Angle:  168.83447
Body:  172.0998
False
squat count: 0

0: 384x640 1 person, 135.8ms
Speed: 5.4ms preprocess, 135.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
Form:  True
Angle:  168.68599
Body:  172.84921
False
squat count: 0
0: 384x640 1 person, 147.8ms
Speed: 3.0ms preprocess, 147.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Form:  True
Angle:  168.32118
Body:  173.49422
False
squat count: 0
0: 384x640 1 person, 136.6ms
Speed: 3.3ms preprocess, 136.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Form:  True
Angle:  168.01436
Body:  174.55547
False
squat count: 0
0: 384x640 1 person, 129.6ms
Speed: 3.2ms preprocess, 129.6ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)
Form:  True
Angle:  167.87115
Body:  175.28714
False
squat count: 0
0: 384x640 1 person, 132.

<img src="picture/Inference 2024-07-10 16-40-03.gif" alt="SQUAT Angle" width="600" height="400">

Grace a la puissance de YOLOv8-Pose, nous avons développé une application de comptage d'exercices qui va au-delà de la simple mesure des répétitions. Notre approche évalue également la précision des mouvements réalisés, offrant ainsi aux utilisateurs un moyen plus engageant et efficace de suivre leurs progrès en fitness. Cette application fournit des informations précieuses et des retours en temps réel, permettant aux utilisateurs d'ajuster leur technique pour optimiser leurs performances.

Actuellement démontrée avec la fonctionnalité de comptage de squats, notre application est également capable de compter les pompes et les tractions. En exploitant la nature modulaire de la base de code, nous avons facilité l'extension de cette fonctionnalité à d'autres types d'exercices. Cela nécessite simplement l'ajustement de la sélection des points clés spécifiques à chaque exercice et des seuils d'angle correspondants.

Cette approche flexible et évolutive permet à notre application de répondre aux besoins variés des utilisateurs, qu'ils soient débutants cherchant à perfectionner leur technique ou athlètes confirmés souhaitant surveiller et améliorer leur performance physique de manière détaillée et précise. En intégrant la technologie de détection de pose avancée à une interface utilisateur conviviale, nous visons à encourager une pratique sportive plus informée et efficace pour tous.